<a href="https://colab.research.google.com/github/deitar/jirawiki2docx/blob/experiment/jira_to_weekly_report_api_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-docx
!pip install jira
!pip install webcolors

In [2]:
!pip install -i https://test.pypi.org/simple/ jirawiki2docx==1.0.11

Looking in indexes: https://test.pypi.org/simple/


In [3]:
from jirawiki2docx import JiraWiki2Docx

In [4]:
import os
import json
from docx import Document
from docx.shared import Inches
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_ALIGN_VERTICAL
from docx.shared import Inches, RGBColor, Pt, Cm
from docx.enum.text import WD_LINE_SPACING
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from itertools import groupby
from datetime import datetime
import numpy as np
import pandas as pd
from jira import JIRA


#### Generic Functions

In [5]:
def group_list_of_dictionaries(input_list, key):
  # Sort the list of dictionaries by the specified key
  sorted_list = sorted(input_list, key=lambda x: x[key])
  # Group the dictionaries based on the specified key
  grouped_dict = {}
  for k, g in groupby(sorted_list, key=lambda x: x[key]):
      grouped_dict[k] = list(g)
  return grouped_dict


def convert_date_to_jira_format(dt):
  return datetime.strftime(dt, '%Y-%m-%d')


def convert_date_to_weekly_report_format(dt):
  return datetime.strftime(dt, '%d/%m/%Y')

#### Word Doc Functions

In [6]:
def set_page_margins(document, top_margin, bottom_margin, left_margin, right_margin):
  #changing the page margins
  sections = document.sections
  for section in sections:
    section.top_margin = top_margin
    section.bottom_margin = bottom_margin
    section.left_margin = left_margin
    section.right_margin = right_margin


def set_col_widths(table, *width_args):
    for row in table.rows:
        for idx, width in enumerate(width_args):
            row.cells[idx].width = width


def build_table_row_from_issue(table, issue):
  title = issue.get('summary','') or ''
  beneficiaries = issue.get('beneficiaries','') or ''
  description = issue.get('description','') or ''
  impact = issue.get('impact','') or ''
  status = issue.get('status','') or ''
  completion_stage = issue.get('completion_stage','') or ''

  new_row = table.add_row().cells
  new_row[0].text = title
  new_row[1].text = '\n'.join(beneficiaries)
  JiraWiki2Docx(description, new_row[2]).parseJira2Docx()
  # new_row[2].text = description

  if impact is not None and impact.strip() != "":
    run2 = new_row[2].add_paragraph('Impact').runs[0]
    run2.font.bold = True
    JiraWiki2Docx(impact, new_row[2]).parseJira2Docx()
    # new_row[2].add_paragraph(impact)

  new_row[3].text = f'{status.title()} ({completion_stage})' if completion_stage and completion_stage.lower()!='completed' else status.title()


#### Jira Wrapper Functions

In [7]:
def get_issue_region(beneficiary_company):
  if beneficiary_company:
    beneficiary_company = beneficiary_company[0] if type(beneficiary_company)==list else beneficiary_company
    if beneficiary_company in ['GTBank Nigeria','HoldCo','GTFM','GTPM']:
      return REPORT_REGIONS[0]
    if beneficiary_company in ['GTBank Ghana']:
      return REPORT_REGIONS[1]
    elif beneficiary_company in ['GTBank Rwanda','GTBank Kenya','GTBank Tanzania','GTBank Uganda']:
      return REPORT_REGIONS[2]

def convert_object_jira_to_string(prop):
  if type(prop).__name__ in ['Project','Status']:
    return prop.name
  if type(prop).__name__=='CustomFieldOption':
    return prop.value
  return prop

def get_custom_field_value(issue, field_name):
  prop_value = getattr(issue.fields, nameMap.get(field_name), None)
  if type(prop_value)==list:
    prop_value = [convert_object_jira_to_string(p) for p in prop_value]
  else:
    prop_value = convert_object_jira_to_string(prop_value)
  return prop_value

In [8]:
# https://jiraone.readthedocs.io/en/latest/apis.html#jiraone.access.EndPoints.issue_attachments

### Configure and Login to Jira

In [9]:
REPORT_REGIONS = ['Nigeria', 'West Africa', 'East Africa']

In [10]:
REPORT_FROM = datetime(2023,6,19)
REPORT_TO = datetime(2023,6,23)

In [11]:
# Set query criteria
JQL = f'"Exclude from weekly report?[Dropdown]" = No AND ("Completed Date[Date]" >= {convert_date_to_jira_format(REPORT_FROM)} AND "Completed Date[Date]" <= {convert_date_to_jira_format(REPORT_TO)} AND status = Completed) OR (status in ("IN PROGRESS")) order by "Start date[Date]" DESC'

In [12]:
# Get credentials and login to Jira
cred_file = "config.json"
config = json.load(open(cred_file))
jira = JIRA(config['url'], basic_auth=(config['user'], config['password']))

In [13]:
# Fetch all fields
allfields = jira.fields()
# Make a map from field name -> field id
nameMap = {field['name']:field['id'] for field in allfields}

In [14]:
# extract issues
issues = jira.search_issues(JQL, maxResults=100)

In [15]:
processed_issues = []
for issue in issues:
  new_issue = {}

  # Extracting fields from the current issue
  new_issue['project_name'] = get_custom_field_value(issue, 'Project')
  new_issue['summary'] = get_custom_field_value(issue, 'Summary')
  new_issue['beneficiaries'] = get_custom_field_value(issue, 'Beneficiary Team(s)')
  new_issue['description'] = get_custom_field_value(issue, 'Description')
  new_issue['impact'] = get_custom_field_value(issue, 'Impact')
  new_issue['status'] = get_custom_field_value(issue, 'Status')
  new_issue['country'] = get_custom_field_value(issue, 'Beneficiary Country')
  new_issue['beneficiary_company'] = get_custom_field_value(issue, 'Beneficiary Company')
  new_issue['report_section'] = get_custom_field_value(issue, 'Report Section')
  new_issue['completion_stage'] = get_custom_field_value(issue, 'Completion stage (%)')

  issue_type = get_custom_field_value(issue, 'Issue Type')

  if str(issue_type) == 'Sub-task':
    issue_parent_key = get_custom_field_value(issue, 'Parent').key
    parent_issue = jira.issue(issue_parent_key)
    # Updating fields from the parent issue if they are missing in the current issue
    new_issue['beneficiaries'] = new_issue['beneficiaries'] or get_custom_field_value(parent_issue, 'Beneficiary Team(s)')
    new_issue['description'] = new_issue['description'] or get_custom_field_value(parent_issue, 'Description')
    new_issue['impact'] = new_issue['impact'] or get_custom_field_value(parent_issue, 'Impact')
    new_issue['country'] = new_issue['country'] or get_custom_field_value(parent_issue, 'Beneficiary Country')
    new_issue['beneficiary_company'] = new_issue['beneficiary_company'] or get_custom_field_value(parent_issue, 'Beneficiary Company')
    new_issue['report_section'] = new_issue['report_section'] or get_custom_field_value(parent_issue, 'Report Section')
    new_issue['completion_stage'] = new_issue['completion_stage'] or get_custom_field_value(parent_issue, 'Completion stage (%)')

  new_issue['report_section'] = new_issue['report_section'] if new_issue['report_section'] else new_issue['project_name']
  new_issue['report_region'] = get_issue_region(new_issue['beneficiary_company'])
  # print(new_issue)
  processed_issues.append(new_issue)

#### Export Jira Issues

In [16]:
# processed_issues

In [17]:
# processed_issues_df

In [18]:
print('Number of issues: ', len(processed_issues))

Number of issues:  23


In [19]:
# Create word document
# doc = Document('weekly report template.docx')
doc = Document()

# Set the default styles for the document
DOC_FONT = 'Century Gothic'

# Set document font by modifying the Normal style
style = doc.styles['Normal']
font = style.font
font.name = DOC_FONT
DOC_PRIMARY_COLOR = RGBColor(222, 74, 9)

# Set document margins
set_page_margins(doc, Cm(1), Cm(1), Cm(1), Cm(1))

LOGO_PATH = 'logo.png'
LOGO_WIDTH = Cm(1.3)

# Create a table with one row and two columns
table = doc.add_table(rows=1, cols=2)

# Set alignment to cell
# table.cell(0, 0).vertical_alignment = WD_ALIGN_VERTICAL.CENTER

# Add image to the first column
table.cell(0, 0).paragraphs[0].add_run().add_picture(LOGO_PATH, width = LOGO_WIDTH)

# Add the text lines to the cell
# line 1
para1 = table.cell(0, 1).paragraphs[0]
para1.paragraph_format.line_spacing_rule = WD_LINE_SPACING.SINGLE
para1.paragraph_format.space_after = Pt(0)
run1 = para1.add_run('Group Data Analytics Weekly Report')
run1.font.size = Pt(16)
# line 2
para2 = table.cell(0, 1).add_paragraph(f'Activities for the Week ({convert_date_to_weekly_report_format(REPORT_FROM)} - {convert_date_to_weekly_report_format(REPORT_TO)})')
run2 = para2.runs[0]
run2.font.color.rgb = DOC_PRIMARY_COLOR # Set orange color
run2.font.size = Pt(14) # Set font size to 12 points
run2.font.bold = True

# Set the width of the first column
set_col_widths(table, Cm(0.5), Cm(20))

# Add empty lines
doc.add_paragraph("")
doc.add_paragraph("")

In [20]:
for region in REPORT_REGIONS:
  print('********',region)
  issues_per_region = filter(lambda x:x['report_region']==region, processed_issues)

  # write region to doc
  doc.add_paragraph(region)

  # intitialize table for current region
  table = doc.add_table(rows=1, cols=4)
  # table.style = 'Weekly Report' # Apply a table style if needed
  table.style = 'Table Grid' # Apply a table style if needed

  # write table headers
  table.cell(0, 0).text = "TASK/PROJECT"
  table.cell(0, 1).text = "BENEFICIARY"
  table.cell(0, 2).text = "DESCRIPTION"
  table.cell(0, 3).text = "STATUS"

  # split the issues for each region by report sections
  issues_per_report_section = group_list_of_dictionaries(issues_per_region, 'report_section')
  for rep_sect, issues in issues_per_report_section.items():
    print('----',rep_sect)

    # write report section
    new_row = table.add_row().cells
    merged_row = new_row[0].merge(new_row[1]).merge(new_row[2]).merge(new_row[3])
    merged_run = merged_row.paragraphs[0].add_run()
    merged_run.text = rep_sect
    merged_run.font.bold = True
    shading_elm = parse_xml(r'<w:shd {} w:fill="F3F4F3"/>'.format(nsdecls('w')))
    merged_row._tc.get_or_add_tcPr().append(shading_elm)

    for issue in issues: # iterate over report_section
      print(issue)
      build_table_row_from_issue(table, issue)

  # set task table column widths
  set_col_widths(table, Cm(1), Cm(4), Cm(10), Cm(3))

  # add extra lines after each region
  doc.add_paragraph('')
  doc.add_paragraph('')

******** Nigeria
---- Awareness & Analysis
{'project_name': 'Management/Analysis Reports', 'summary': 'Card Usage Pre & Post Cash Scarcity ', 'beneficiaries': ['Digital_Banking'], 'description': 'Carried out an {color:#ff5630}analysis to {color}{color:#fff0b3}identify{color}{color:#ff5630} c{color}{color:#ff5630}*ustome*{color}{color:#ff5630}rs who{color} carried out POS/WEB (card) {color:#bf2600}transactions {color}during the period of +^*-_cash scarcity but subsequently_-*^+ stopped{color:#00b8d9} transacting{color} on this *channel and map out the* *_transactional_* *behaviour* of _these_ *_customers_* _prior to and during_ cash ^sca^rcity. testing _some_ functions of wiki formating.\n\n*Definition of Terms*\n\n* Cash scarcity period: ~November~ 2022 to March 2023 (150 days) \n* 6 months prior to cash scarcity: May 2022 to October 2022 (180 days) \n* Period after cash scarcity: April 2023 to May 2023 (60 days)\n\n*Key Highlights:*\n\n* Customers transactional behaviour before, durin

In [21]:
doc.save('output.docx')